In [1]:
import networkx as nx
import matplotlib as plt
import numpy as np
import copy

In [4]:
# for testing the random graph generating function
g_ws = nx.watts_strogatz_graph(n = 4000, k = 57,p=0.5)
g_bs = nx.barabasi_albert_graph(n = 4000, m = 6)

In [5]:
# States
HEALTH = 0
TRANSMISSION = 1
IMMUNE = 2 # immune & inactive

# parameter
p1 = 0.5 # health to transmission
p2 = 0.55 # immune to transmission
# a = 0.45 # user growth rate per month
# b = 0.3 # inactive rate per month

class social_network():
    def __init__(self,  alpha=0.1,beta=0.3, n = 4000,name = "ws",):
        if name == "ws":
            self.graph = nx.watts_strogatz_graph(n, k = 57, p=0.5)
        elif name == "ba":
            self.graph = nx.barabasi_albert_graph(n, m = 6)
        
        
        init_dist=np.random.choice([TRANSMISSION,IMMUNE,HEALTH],
                         size = n,p=(alpha,beta,1-alpha-beta))
        
        # list of nodes group by status
        self.hea_list = np.where(init_dist==HEALTH)[0].tolist()
        self.trans_list = np.where(init_dist==TRANSMISSION)[0].tolist()
        self.im_list = np.where(init_dist==IMMUNE)[0].tolist()
        
        for i in range(0,n):
            self.graph.node[i]['status'] = init_dist[i]

In [6]:
# initializing a social network
G=social_network(name ="ba")
g = G.graph
trans_list = G.trans_list
hea_list = G.hea_list
im_list = G.im_list
nodes = g.nodes(data = True)

In [8]:
# finish changing the transmision nodes' status
for nt in trans_list:

    nei_list = np.array(g.neighbors(nt))
    nei_sta = np.array([g.node[nei]['status'] 
                        for nei in nei_list])
    # a = copy.deepcopy(nei_sta)

    heal_nei = nei_list[np.where(nei_sta == HEALTH)[0]]
    spreaded_nei = heal_nei[np.where(np.random.uniform(0,1,(heal_nei.shape[0],)) 
                              < p1)[0]]

    if spreaded_nei.size!=0:    # if spread to nei
        G.trans_list.remove(nt)
        G.im_list.append(nt)
        
        g.node[nt]['status'] = IMMUNE
        
        
        
        for sn in spreaded_nei:
            G.hea_list.remove(sn)
            G.trans_list.append(sn)
            
            
            g.node[sn]['status'] = TRANSMISSION